## GoogleNews
### AZ BNT 莫德納 高端

### 載入套件

In [1]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker
from snownlp import SnowNLP
import re
import emoji
from collections import Counter
import jieba
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
import time
from fake_useragent import UserAgent

ua = UserAgent() # 產生一個 user agent
user_agent = ua.random # 隨機產生一個 user agent字串

vaccine_categories = ['AZ','BNT','莫德納','高端']


### 將四種疫苗的標題與網址爬下來

In [128]:
%%time

# CNA
url_AZ = 'https://news.google.com/search?q=AZ%E7%96%AB%E8%8B%97%20site%3Awww.cna.com.tw%20%22AZ%E7%96%AB%E8%8B%97%22&hl=zh-TW&gl=TW&ceid=TW%3Azh-Hant'
url_BNT = 'https://news.google.com/search?q=BNT%E7%96%AB%E8%8B%97%20site%3Awww.cna.com.tw%20%22BNT%E7%96%AB%E8%8B%97%22&hl=zh-TW&gl=TW&ceid=TW%3Azh-Hant'
url_Moderna = 'https://news.google.com/search?q=%E8%8E%AB%E5%BE%B7%E7%B4%8D%E7%96%AB%E8%8B%97%20site%3Awww.cna.com.tw%20%22%E8%8E%AB%E5%BE%B7%E7%B4%8D%E7%96%AB%E8%8B%97%22&hl=zh-TW&gl=TW&ceid=TW%3Azh-Hant'
url_MVC = 'https://news.google.com/search?q=%E9%AB%98%E7%AB%AF%E7%96%AB%E8%8B%97%20site%3Awww.cna.com.tw%20%22%E9%AB%98%E7%AB%AF%E7%96%AB%E8%8B%97%22&hl=zh-TW&gl=TW&ceid=TW%3Azh-Hant'

# # yahoo
# url_yahoo_AZ = 'https://news.google.com/search?q=AZ%E7%96%AB%E8%8B%97%20site%3Atw.yahoo.com%20%22AZ%E7%96%AB%E8%8B%97%22&hl=zh-TW&gl=TW&ceid=TW%3Azh-Hant'
# url_yahoo_BNT = 'https://news.google.com/search?q=BNT%E7%96%AB%E8%8B%97%20site%3Atw.yahoo.com%20%22BNT%E7%96%AB%E8%8B%97%22&hl=zh-TW&gl=TW&ceid=TW%3Azh-Hant'
# url_yahoo_Moderna = 'https://news.google.com/search?q=%E8%8E%AB%E5%BE%B7%E7%B4%8D%E7%96%AB%E8%8B%97%20site%3Atw.yahoo.com%20%22%E8%8E%AB%E5%BE%B7%E7%B4%8D%E7%96%AB%E8%8B%97%22&hl=zh-TW&gl=TW&ceid=TW%3Azh-Hant'
# url_yahoo_MVC = 'https://news.google.com/search?q=%E9%AB%98%E7%AB%AF%E3%80%81MVC%20site%3Atw.yahoo.com%20%22%E9%AB%98%E7%AB%AF%E7%96%AB%E8%8B%97%22&hl=zh-TW&gl=TW&ceid=TW%3Azh-Hant'

# china
url_china_AZ = 'https://news.google.com/search?q=AZ%E7%96%AB%E8%8B%97%20site%3Awww.chinatimes.com%20%22AZ%E7%96%AB%E8%8B%97%22&hl=zh-TW&gl=TW&ceid=TW%3Azh-Hant'
url_china_BNT = 'https://news.google.com/search?q=BNT%E7%96%AB%E8%8B%97%20site%3Awww.chinatimes.com%20%22BNT%E7%96%AB%E8%8B%97%22&hl=zh-TW&gl=TW&ceid=TW%3Azh-Hant'
url_china_Moderna = 'https://news.google.com/search?q=%E8%8E%AB%E5%BE%B7%E7%B4%8D%E7%96%AB%E8%8B%97%20site%3Awww.chinatimes.com%20%22%E8%8E%AB%E5%BE%B7%E7%B4%8D%E7%96%AB%E8%8B%97%22&hl=zh-TW&gl=TW&ceid=TW%3Azh-Hant'
url_china_MVC = 'https://news.google.com/search?q=%E9%AB%98%E7%AB%AF%E7%96%AB%E8%8B%97%20site%3Awww.chinatimes.com%20%22%E9%AB%98%E7%AB%AF%E7%96%AB%E8%8B%97%22&hl=zh-TW&gl=TW&ceid=TW%3Azh-Hant'


ua = UserAgent() # 產生一個 user agent
user_agent = ua.random # 隨機產生一個 user agent字串

# 發送get請求
headers = {'user-agent': user_agent}

# CNA
r_AZ = requests.get(url_AZ, headers=headers)
web_content_AZ  = r_AZ.text
soup_AZ  = BeautifulSoup(web_content_AZ ,'lxml')

r_BNT = requests.get(url_BNT, headers=headers)
web_content_BNT  = r_BNT.text
soup_BNT  = BeautifulSoup(web_content_BNT ,'lxml')

r_Moderna = requests.get(url_Moderna, headers=headers)
web_content_Moderna  = r_Moderna.text
soup_Moderna  = BeautifulSoup(web_content_Moderna ,'lxml')

r_MVC = requests.get(url_MVC, headers=headers)
web_content_MVC  = r_MVC.text
soup_MVC  = BeautifulSoup(web_content_MVC ,'lxml')


# # yahoo
# r_yahoo_AZ = requests.get(url_yahoo_AZ, headers=headers)
# web_content_yahoo_AZ  = r_yahoo_AZ.text
# soup_yahoo_AZ  = BeautifulSoup(web_content_yahoo_AZ ,'lxml')

# r_yahoo_BNT = requests.get(url_yahoo_BNT, headers=headers)
# web_content_yahoo_BNT  = r_yahoo_BNT.text
# soup_yahoo_BNT  = BeautifulSoup(web_content_yahoo_BNT ,'lxml')

# r_yahoo_Moderna = requests.get(url_yahoo_Moderna, headers=headers)
# web_content_yahoo_Moderna  = r_yahoo_Moderna.text
# soup_yahoo_Moderna  = BeautifulSoup(web_content_yahoo_Moderna ,'lxml')

# r_yahoo_MVC = requests.get(url_yahoo_MVC, headers=headers)
# web_content_yahoo_MVC  = r_yahoo_MVC.text
# soup_yahoo_MVC  = BeautifulSoup(web_content_yahoo_MVC ,'lxml')


# china
r_china_AZ = requests.get(url_china_AZ, headers=headers)
web_content_china_AZ  = r_china_AZ.text
soup_china_AZ  = BeautifulSoup(web_content_china_AZ ,'lxml')

r_china_BNT = requests.get(url_china_BNT, headers=headers)
web_content_china_BNT  = r_china_BNT.text
soup_china_BNT  = BeautifulSoup(web_content_china_BNT ,'lxml')

r_china_Moderna = requests.get(url_china_Moderna, headers=headers)
web_content_china_Moderna  = r_china_Moderna.text
soup_china_Moderna  = BeautifulSoup(web_content_china_Moderna ,'lxml')

r_china_MVC = requests.get(url_china_MVC, headers=headers)
web_content_china_MVC  = r_china_MVC.text
soup_china_MVC  = BeautifulSoup(web_content_china_MVC ,'lxml')

CPU times: total: 1.78 s
Wall time: 11.7 s


In [3]:
# CNA_AZ
ua = UserAgent() # 產生一個 user agent
user_agent = ua.random # 隨機產生一個 user agent字串

# 發送get請求
headers = {'user-agent': user_agent}

title_AZ = soup_AZ.find_all('h3', class_='ipQwMb ekueJc RD0gLb')
titles_AZ = [t.find('a').text for t in title_AZ]
titles_AZ

newUrls_AZ = [requests.get(t.find('a')['href'].replace('.','https://news.google.com',1), headers=headers).url for t in title_AZ]
newUrls_AZ

time.sleep(2)

In [129]:
# # yahoo_AZ
# ua = UserAgent() # 產生一個 user agent
# user_agent = ua.random # 隨機產生一個 user agent字串

# # 發送get請求
# headers = {'user-agent': user_agent}

# title_yahoo_AZ = soup_yahoo_AZ.find_all('h3', class_='ipQwMb ekueJc RD0gLb')
# titles_yahoo_AZ = [t.find('a').text for t in title_yahoo_AZ]
# titles_yahoo_AZ

# newUrls_yahoo_AZ = [requests.get(t.find('a')['href'].replace('.','https://news.google.com',1), headers=headers).url for t in title_yahoo_AZ]
# newUrls_yahoo_AZ

# time.sleep(2)

In [40]:
# china_AZ
ua = UserAgent() # 產生一個 user agent
user_agent = ua.random # 隨機產生一個 user agent字串

# 發送get請求
headers = {'user-agent': user_agent}

title_china_AZ = soup_china_AZ.find_all('h3', class_='ipQwMb ekueJc RD0gLb')
titles_china_AZ = [t.find('a').text for t in title_china_AZ]
titles_china_AZ

newUrls_china_AZ = [requests.get(t.find('a')['href'].replace('.','https://news.google.com',1), headers=headers).url for t in title_china_AZ]
newUrls_china_AZ

time.sleep(2)

In [4]:
# CNA_BNT
ua = UserAgent() # 產生一個 user agent
user_agent = ua.random # 隨機產生一個 user agent字串

# 發送get請求
headers = {'user-agent': user_agent}

title_BNT = soup_BNT.find_all('h3', class_='ipQwMb ekueJc RD0gLb')
titles_BNT = [t.find('a').text for t in title_BNT]
titles_BNT

newUrls_BNT = [requests.get(t.find('a')['href'].replace('.','https://news.google.com',1), headers=headers).url for t in title_BNT]
newUrls_BNT

time.sleep(2)

In [33]:
# # yahoo_BNT
# ua = UserAgent() # 產生一個 user agent
# user_agent = ua.random # 隨機產生一個 user agent字串

# # 發送get請求
# headers = {'user-agent': user_agent}

# title_yahoo_BNT = soup_yahoo_BNT.find_all('h3', class_='ipQwMb ekueJc RD0gLb')
# titles_yahoo_BNT = [t.find('a').text for t in title_yahoo_BNT]
# titles_yahoo_BNT

# newUrls_yahoo_BNT = [requests.get(t.find('a')['href'].replace('.','https://news.google.com',1), headers=headers).url for t in title_yahoo_BNT]
# newUrls_yahoo_BNT

# time.sleep(2)

In [42]:
# china_BNT
ua = UserAgent() # 產生一個 user agent
user_agent = ua.random # 隨機產生一個 user agent字串

# 發送get請求
headers = {'user-agent': user_agent}

title_china_BNT = soup_china_BNT.find_all('h3', class_='ipQwMb ekueJc RD0gLb')
titles_china_BNT = [t.find('a').text for t in title_china_BNT]
titles_china_BNT

newUrls_china_BNT = [requests.get(t.find('a')['href'].replace('.','https://news.google.com',1), headers=headers).url for t in title_china_BNT]
newUrls_china_BNT

time.sleep(2)

In [5]:
# CNA_Moderna
ua = UserAgent() # 產生一個 user agent
user_agent = ua.random # 隨機產生一個 user agent字串

# 發送get請求
headers = {'user-agent': user_agent}

title_Moderna = soup_Moderna.find_all('h3', class_='ipQwMb ekueJc RD0gLb')
titles_Moderna = [t.find('a').text for t in title_Moderna]
titles_Moderna

newUrls_Moderna = [requests.get(t.find('a')['href'].replace('.','https://news.google.com',1), headers=headers).url for t in title_Moderna]
newUrls_Moderna

time.sleep(2)

In [36]:
# # yahoo_Moderna
# ua = UserAgent() # 產生一個 user agent
# user_agent = ua.random # 隨機產生一個 user agent字串

# # 發送get請求
# headers = {'user-agent': user_agent}

# title_yahoo_Moderna = soup_yahoo_Moderna.find_all('h3', class_='ipQwMb ekueJc RD0gLb')
# titles_yahoo_Moderna = [t.find('a').text for t in title_yahoo_Moderna]
# titles_yahoo_Moderna

# newUrls_yahoo_Moderna = [requests.get(t.find('a')['href'].replace('.','https://news.google.com',1), headers=headers).url for t in title_yahoo_Moderna]
# newUrls_yahoo_Moderna

# time.sleep(2)

In [44]:
# china_Moderna
ua = UserAgent() # 產生一個 user agent
user_agent = ua.random # 隨機產生一個 user agent字串

# 發送get請求
headers = {'user-agent': user_agent}

title_china_Moderna = soup_china_Moderna.find_all('h3', class_='ipQwMb ekueJc RD0gLb')
titles_china_Moderna = [t.find('a').text for t in title_china_Moderna]
titles_china_Moderna

newUrls_china_Moderna = [requests.get(t.find('a')['href'].replace('.','https://news.google.com',1), headers=headers).url for t in title_china_Moderna]
newUrls_china_Moderna

time.sleep(2)

In [6]:
# CNA_MVC
ua = UserAgent() # 產生一個 user agent
user_agent = ua.random # 隨機產生一個 user agent字串

# 發送get請求
headers = {'user-agent': user_agent}

title_MVC = soup_MVC.find_all('h3', class_='ipQwMb ekueJc RD0gLb')
titles_MVC = [t.find('a').text for t in title_MVC]
titles_MVC

newUrls_MVC = [requests.get(t.find('a')['href'].replace('.','https://news.google.com',1), headers=headers).url for t in title_MVC]
newUrls_MVC

time.sleep(2)

In [38]:
# # yahoo_MVC
# ua = UserAgent() # 產生一個 user agent
# user_agent = ua.random # 隨機產生一個 user agent字串

# # 發送get請求
# headers = {'user-agent': user_agent}

# title_yahoo_MVC = soup_yahoo_MVC.find_all('h3', class_='ipQwMb ekueJc RD0gLb')
# titles_yahoo_MVC = [t.find('a').text for t in title_yahoo_MVC]
# titles_yahoo_MVC

# newUrls_yahoo_MVC = [requests.get(t.find('a')['href'].replace('.','https://news.google.com',1), headers=headers).url for t in title_yahoo_MVC]
# newUrls_yahoo_MVC

# time.sleep(2)

In [46]:
# china_MVC
ua = UserAgent() # 產生一個 user agent
user_agent = ua.random # 隨機產生一個 user agent字串

# 發送get請求
headers = {'user-agent': user_agent}

title_china_MVC = soup_china_MVC.find_all('h3', class_='ipQwMb ekueJc RD0gLb')
titles_china_MVC = [t.find('a').text for t in title_china_MVC]
titles_china_MVC

newUrls_china_MVC = [requests.get(t.find('a')['href'].replace('.','https://news.google.com',1), headers=headers).url for t in title_china_MVC]
newUrls_china_MVC

time.sleep(2)

In [141]:
titles_vaccine = titles_AZ + titles_BNT + titles_Moderna + titles_MVC + titles_china_AZ + titles_china_BNT + titles_china_Moderna + titles_china_MVC
newUrls_vaccine = newUrls_AZ + newUrls_BNT + newUrls_Moderna + newUrls_MVC + newUrls_china_AZ + newUrls_china_BNT + newUrls_china_Moderna + newUrls_china_MVC

In [142]:
titles_china_vaccine = titles_china_AZ + titles_china_BNT + titles_china_Moderna + titles_china_MVC
newUrls_china_vaccine = newUrls_china_AZ + newUrls_china_BNT + newUrls_china_Moderna + newUrls_china_MVC

In [143]:
df = pd.DataFrame(
{
    'title': titles_vaccine,
    'links': newUrls_vaccine,
})
 
df

,title,links
0,澎湖確診數升 兒童疫苗接種率已逾7成,https://news.google.com/articles/CBMiMmh0dHBzO...
1,北市50多歲女打AZ疫苗後死亡 獲救濟金600萬史上最高,https://news.google.com/articles/CBMiMmh0dHBzO...
2,日本評估打不完 取消4000萬劑AZ疫苗訂單,https://news.google.com/articles/CBMiMmh0dHBzO...
3,AZ疫苗發明人警告 下一場大流行可能更致命,https://news.google.com/articles/CBMiN2h0dHBzO...
4,AZ疫苗第2劑本週開放50至64歲預約 可望雙十連假前開打,https://news.google.com/articles/CBMiN2h0dHBzO...
...,...,...
795,賴清德出訪接觸者陸續傳確診 柯P：打高端不見得沒效 但密醫上法庭必敗,https://news.google.com/articles/CAIiEKyuOudQM...
796,打高端不能入境日本？各國打疫苗免隔離規定一次看懂,https://news.google.com/articles/CAIiEFeOOeMUD...
797,有望打高端疫苗 網拱焦糖「快取消AZ」他笑回：我的身體我作主,https://news.google.com/articles/CAIiEJVKY1u_b...
798,中時社論》高端與台獨 民進黨夢工廠,https://news.google.com/articles/CBMiOGh0dHBzO...


In [144]:
df_china = pd.DataFrame(
{
    'title': titles_china_vaccine,
    'links': newUrls_china_vaccine,
})
 
df_china

,title,links
0,AZ疫苗3個月後保護力下滑 國內2因素 堅持5個月打第3劑,https://news.google.com/articles/CBMiPWh0dHBzO...
1,第11期疫苗來了！提供AZ疫苗第二劑及BNT疫苗第一劑接種,https://news.google.com/articles/CAIiEEoJN4ftM...
2,719沒勾AZ超後悔？她一招讓全家一天內打到疫苗,https://news.google.com/articles/CAIiEOS792oi8...
3,海納百川》即期AZ疫苗開打 民眾自求多福（夏學理）,https://news.google.com/articles/CBMiOGh0dHBzO...
4,AZ疫苗研發者呼籲：不是每個人都要打第三劑,https://news.google.com/articles/CAIiEE1zxuWiD...
...,...,...
395,賴清德出訪接觸者陸續傳確診 柯P：打高端不見得沒效 但密醫上法庭必敗,https://news.google.com/articles/CAIiEKyuOudQM...
396,打高端不能入境日本？各國打疫苗免隔離規定一次看懂,https://news.google.com/articles/CAIiEFeOOeMUD...
397,有望打高端疫苗 網拱焦糖「快取消AZ」他笑回：我的身體我作主,https://news.google.com/articles/CAIiEJVKY1u_b...
398,中時社論》高端與台獨 民進黨夢工廠,https://news.google.com/articles/CBMiOGh0dHBzO...


### 整合4種疫苗

In [51]:
# AZ整合 CNA

articleAll_AZs = []
articleContent = []

for i in range(0, 99):

    url = df['links'][i]
    r = requests.get(url)
    web_content = r.text
    soup = BeautifulSoup(web_content,'lxml')

    # Find content
    content = soup.find('div',{'class':"paragraph"})

    # Remove unnecessary content

    # 額外插入的延伸閱讀區塊-->移除之
    try:
        content.find('div', {'class':"paragraph moreArticle"}).decompose()
    except:
        pass
    # 額外插入的訂閱連結-->移除之
    try:
        content.find('div', class_="SubscriptionInner mySubscriptionInner").decompose()
    except:
        pass

    # 額外插入的圖片與文字說明區塊-->移除之
    try:
        for media in content.findAll('div', {'class':"media"}):
            media.decompose()
    except:
        pass

    # Remove unnecessary content
    cont = content.text
    cont = re.sub('（中央社記者.+?）','', cont)
    cont = re.sub('（編輯：.*','', cont) 
    cont = re.sub('（譯者：.*','', cont) 
    cont = re.sub('（中央社\w*）','', cont)

    articleAll_AZs.append(cont) 

In [119]:
# AZ整合 china

articleAll_china_AZs = []
articleContent = []

for i in range(0, 100):

    url = df_china['links'][i]
    r = requests.get(url)
    web_content = r.text
    soup = BeautifulSoup(web_content,'lxml')

    # Find content
    content = soup.find('div',{'class':'article-body'})


    try:
        content.find('div', {'class':"paragraph moreArticle"}).decompose()
    except:
        pass
    try:
        content.find('div', class_="SubscriptionInner mySubscriptionInner").decompose()
    except:
        pass

    try:
        for media in content.findAll('div', {'class':"media"}):
            media.decompose()
    except:
        pass

    cont = content.text
    cont = re.sub('（中央社記者.+?）','', cont)
    cont = re.sub('（編輯：.*','', cont) 
    cont = re.sub('（譯者：.*','', cont) 
    cont = re.sub('（中央社\w*）','', cont) 
    cont = re.sub('\n','', cont)
    cont = re.sub('#\w*','', cont)

    articleAll_china_AZs.append(cont)

In [76]:
# BNT整合 CNA

articleAll_BNTs = []
articleContent = []

for i in range(100, 199):

    url = df['links'][i]
    r = requests.get(url)
    web_content = r.text
    soup = BeautifulSoup(web_content,'lxml')

    # Find content
    content = soup.find('div',{'class':"paragraph"})

    # Remove unnecessary content

    # 額外插入的延伸閱讀區塊-->移除之
    try:
        content.find('div', {'class':"paragraph moreArticle"}).decompose()
    except:
        pass
    # 額外插入的訂閱連結-->移除之
    try:
        content.find('div', class_="SubscriptionInner mySubscriptionInner").decompose()
    except:
        pass

    # 額外插入的圖片與文字說明區塊-->移除之
    try:
        for media in content.findAll('div', {'class':"media"}):
            media.decompose()
    except:
        pass

    # Remove unnecessary content
    cont = content.text
    cont = re.sub('（中央社記者.+?）','', cont)
    cont = re.sub('（編輯：.*','', cont) 
    cont = re.sub('（譯者：.*','', cont) 
    cont = re.sub('（中央社\w*）','', cont)

    articleAll_BNTs.append(cont) 

In [122]:
# BNT整合 china

articleAll_china_BNTs = []
articleContent = []

for i in range(101, 201):

    url = df_china['links'][i]
    r = requests.get(url)
    web_content = r.text
    soup = BeautifulSoup(web_content,'lxml')

    # Find content
    content = soup.find('div',{'class':'article-body'})


    try:
        content.find('div', {'class':"paragraph moreArticle"}).decompose()
    except:
        pass
    try:
        content.find('div', class_="SubscriptionInner mySubscriptionInner").decompose()
    except:
        pass

    try:
        for media in content.findAll('div', {'class':"media"}):
            media.decompose()
    except:
        pass

    cont = content.text
    cont = re.sub('（中央社記者.+?）','', cont)
    cont = re.sub('（編輯：.*','', cont) 
    cont = re.sub('（譯者：.*','', cont) 
    cont = re.sub('（中央社\w*）','', cont) 
    cont = re.sub('\n','', cont)
    cont = re.sub('#\w*','', cont)

    articleAll_china_BNTs.append(cont)

In [14]:
# 莫德納整合 CNA

articleAll_Modernas = []
articleContent = []

for i in range(200, 299):

    url = df['links'][i]
    r = requests.get(url)
    web_content = r.text
    soup = BeautifulSoup(web_content,'lxml')

    # Find content
    content = soup.find('div',{'class':"paragraph"})

    # Remove unnecessary content

    # 額外插入的延伸閱讀區塊-->移除之
    try:
        content.find('div', {'class':"paragraph moreArticle"}).decompose()
    except:
        pass
    # 額外插入的訂閱連結-->移除之
    try:
        content.find('div', class_="SubscriptionInner mySubscriptionInner").decompose()
    except:
        pass

    # 額外插入的圖片與文字說明區塊-->移除之
    try:
        for media in content.findAll('div', {'class':"media"}):
            media.decompose()
    except:
        pass

    # Remove unnecessary content
    cont = content.text
    cont = re.sub('（中央社記者.+?）','', cont)
    cont = re.sub('（編輯：.*','', cont) 
    cont = re.sub('（譯者：.*','', cont) 
    cont = re.sub('（中央社\w*）','', cont)

    articleAll_Modernas.append(cont) 

In [124]:
# 莫德納整合 china

articleAll_china_Modernas = []
articleContent = []

for i in range(202, 302):

    url = df_china['links'][i]
    r = requests.get(url)
    web_content = r.text
    soup = BeautifulSoup(web_content,'lxml')

    # Find content
    content = soup.find('div',{'class':'article-body'})


    try:
        content.find('div', {'class':"paragraph moreArticle"}).decompose()
    except:
        pass
    try:
        content.find('div', class_="SubscriptionInner mySubscriptionInner").decompose()
    except:
        pass

    try:
        for media in content.findAll('div', {'class':"media"}):
            media.decompose()
    except:
        pass

    cont = content.text
    cont = re.sub('（中央社記者.+?）','', cont)
    cont = re.sub('（編輯：.*','', cont) 
    cont = re.sub('（譯者：.*','', cont) 
    cont = re.sub('（中央社\w*）','', cont) 
    cont = re.sub('\n','', cont)
    cont = re.sub('#\w*','', cont)

    articleAll_china_Modernas.append(cont)

In [16]:
# 高端整合 CNA

articleAll_MVCs = []
articleContent = []

for i in range(300, 399):

    url = df['links'][i]
    r = requests.get(url)
    web_content = r.text
    soup = BeautifulSoup(web_content,'lxml')

    # Find content
    content = soup.find('div',{'class':"paragraph"})

    # Remove unnecessary content

    # 額外插入的延伸閱讀區塊-->移除之
    try:
        content.find('div', {'class':"paragraph moreArticle"}).decompose()
    except:
        pass
    # 額外插入的訂閱連結-->移除之
    try:
        content.find('div', class_="SubscriptionInner mySubscriptionInner").decompose()
    except:
        pass

    # 額外插入的圖片與文字說明區塊-->移除之
    try:
        for media in content.findAll('div', {'class':"media"}):
            media.decompose()
    except:
        pass

    # Remove unnecessary content
    cont = content.text
    cont = re.sub('（中央社記者.+?）','', cont)
    cont = re.sub('（編輯：.*','', cont) 
    cont = re.sub('（譯者：.*','', cont) 
    cont = re.sub('（中央社\w*）','', cont)

    articleAll_MVCs.append(cont) 

In [126]:
# 高端整合 china

articleAll_china_MVCs = []
articleContent = []

for i in range(202, 302):

    url = df_china['links'][i]
    r = requests.get(url)
    web_content = r.text
    soup = BeautifulSoup(web_content,'lxml')

    # Find content
    content = soup.find('div',{'class':'article-body'})


    try:
        content.find('div', {'class':"paragraph moreArticle"}).decompose()
    except:
        pass
    try:
        content.find('div', class_="SubscriptionInner mySubscriptionInner").decompose()
    except:
        pass

    try:
        for media in content.findAll('div', {'class':"media"}):
            media.decompose()
    except:
        pass

    cont = content.text
    cont = re.sub('（中央社記者.+?）','', cont)
    cont = re.sub('（編輯：.*','', cont) 
    cont = re.sub('（譯者：.*','', cont) 
    cont = re.sub('（中央社\w*）','', cont) 
    cont = re.sub('\n','', cont)
    cont = re.sub('#\w*','', cont)

    articleAll_china_MVCs.append(cont)

In [145]:
titles = titles_AZ + titles_china_AZ + titles_BNT + titles_china_BNT + titles_Moderna + titles_china_Moderna + titles_MVC  + titles_china_MVC
contents = articleAll_AZs + articleAll_china_AZs + articleAll_BNTs + articleAll_china_BNTs + articleAll_Modernas + articleAll_china_Modernas + articleAll_MVCs + articleAll_china_MVCs
urls = newUrls_AZ + newUrls_china_AZ + newUrls_BNT + newUrls_china_BNT + newUrls_Moderna + newUrls_china_Moderna + newUrls_MVC + newUrls_china_MVC

In [153]:
categories = []

for i  in range(0, 200):
    categories.append('AZ')

for i  in range(201, 401):
    categories.append('BNT')

for i  in range(402, 602):
    categories.append('莫德納')

for i  in range(603, 803):
    categories.append('高端')

In [162]:
datas = zip( categories, titles, contents, urls)
df_all = pd.DataFrame(list(datas), columns=['categories', 'titles','contents','urls'])
df_all

,categories,titles,contents,urls
0,AZ,澎湖確診數升 兒童疫苗接種率已逾7成,澎湖COVID-19（2019冠狀病毒疾病）確診數攀升，澎湖縣政府今天公布新增182例，還有...,https://news.google.com/articles/CBMiMmh0dHBzO...
1,AZ,北市50多歲女打AZ疫苗後死亡 獲救濟金600萬史上最高,衛福部預防接種受害救濟審議小組公布最新結果，台北市50多歲女性接種AZ疫苗後血栓併血小板低下...,https://news.google.com/articles/CBMiMmh0dHBzO...
2,AZ,日本評估打不完 取消4000萬劑AZ疫苗訂單,日本下訂1億2000萬劑英國阿斯特捷利康（AstraZeneca，簡稱AZ）疫苗，但因日本規...,https://news.google.com/articles/CBMiMmh0dHBzO...
3,AZ,AZ疫苗發明人警告 下一場大流行可能更致命,AZ疫苗研發團隊領導人兼牛津大學教授吉伯特坦言，面對COVID-19變種病毒Omicron，...,https://news.google.com/articles/CBMiN2h0dHBzO...
4,AZ,AZ疫苗第2劑本週開放50至64歲預約 可望雙十連假前開打,指揮中心今天公布，AZ疫苗第2劑改由線上預約，即起開放9月30日前間隔滿10週以上50歲至6...,https://news.google.com/articles/CBMiN2h0dHBzO...
...,...,...,...,...
791,高端,高端次世代新冠疫苗 今向食藥署申請人體試驗,美國在台協會（AIT）今晚在臉書發文證實，美國將贈送150萬劑莫德納疫苗給台灣，預計於明（1...,https://news.google.com/articles/CBMiPWh0dHBzO...
792,高端,高端疫苗難擋Delta？ 高端：對防重症仍有效果,公所人員、里長等人不斷向在場民眾廣播「不用急在今天一定要打」。（陳夢茹攝）桃機群聚案延燒，桃...,https://news.google.com/articles/CAIiEKD0Lu-0Z...
793,高端,新聞透視》從疫苗到快篩 國家隊不落外人田,南韓8月派出代表團向新冠肺炎疫苗廠莫德納（Moderna）催貨，結果很給力。根據韓國媒體報導...,https://news.google.com/articles/CBMiO2h0dHBzO...
794,高端,轟疫苗政治凌駕專業 國民黨要求民進黨道歉,"為因應國際出貨備料，高端疫苗（6547）辦理現金增資，以每股220元發行7,000張股票，預...",https://news.google.com/articles/CAIiEBoxpFsP2...


In [168]:
df_all.to_csv("category_vaccine.csv", sep="|", index=False)

In [192]:

df2 = pd.DataFrame(
{
    'category': ['AZ', 'BNT', '莫德納', '高端'],
    'title':
    'content':
    'sentiment':
    'top_key_freq':
    'tokens':	
    'tokens_v2':
    'entities':
    'token_pos':
    'link':
    
})
 
df2

,category
0,AZ
1,BNT
2,莫德納
3,高端
